In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/biomass/Biomass_History.csv
/kaggle/input/distance/Distance_Matrix.csv


In [2]:
!pip install keras-rl2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 2.2 MB/s eta 0:00:00


In [3]:
import plotly.express as px
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredLogarithmicError,MeanSquaredError,MeanAbsoluteError
from matplotlib import pyplot as plt
from shapely.geometry import Point, MultiPoint
import rtree
from sklearn.neighbors import NearestNeighbors
from gym import Env
from gym.spaces import Box, MultiDiscrete, Dict,Discrete
from shapely.ops import nearest_points
import geopandas as gpd
import seaborn as sns
import random
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
%matplotlib inline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
biomass = pd.read_csv("/kaggle/input/biomass/Biomass_History.csv")
distance = pd.read_csv("/kaggle/input/distance/Distance_Matrix.csv")
# creating point objects, will be useful later on 
points = []
for i, row in biomass.iterrows():
    points.append((float(row['Longitude']),float(row["Latitude"])) )

biomass["Geometries"] = points
biomass["Type"] = "Biomass prod.plants"
print(biomass.head())


   Index  Latitude  Longitude       2010       2011       2012       2013  \
0      0  24.66818   71.33144   8.475744   8.868568   9.202181   6.023070   
1      1  24.66818   71.41106  24.029778  28.551348  25.866415  21.634459   
2      2  24.66818   71.49069  44.831635  66.111168  56.982258  53.003735   
3      3  24.66818   71.57031  59.974419  80.821304  78.956543  63.160561   
4      4  24.66818   71.64994  14.653370  19.327524  21.928144  17.899586   

        2014       2015       2016        2017            Geometries  \
0  10.788374   6.647325   7.387925    5.180296  (71.33144, 24.66818)   
1  34.419411  27.361908  40.431847   42.126945  (71.41106, 24.66818)   
2  70.917908  42.517117  59.181629   73.203232  (71.49069, 24.66818)   
3  93.513924  70.203171  74.536720  101.067352  (71.57031, 24.66818)   
4  19.534035  19.165791  16.531315   26.086885  (71.64994, 24.66818)   

                  Type  
0  Biomass prod.plants  
1  Biomass prod.plants  
2  Biomass prod.plants  
3  B

In [ ]:
distance.iloc[1,19]


In [ ]:
sns.heatmap(biomass.corr(), annot=True)
plt.show()

In [ ]:
fig = px.scatter_mapbox(biomass, lat="Latitude", lon="Longitude",hover_data=["Index"],
                        color="2014", zoom=3, height=400)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show(renderer='iframe')

In [ ]:
#predictions using neural networks

""" First a little data prep. We need to batch the results from 2010 till 2016, the 2017 
results will serve as the target values for the neural network"""

x = biomass[[str(i+2000) for i in range(10,17)]].values
print(x.shape)
y = biomass["2017"].values
print(y.shape)
scaler = StandardScaler()
x = scaler.fit_transform(x)
X_train,X_test,Y_train,Y_test = train_test_split(x,y, test_size = 0.25, random_state = 32)
print (X_train.shape, X_test.shape)


def build_net_no_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu'))
    model.add(tf.keras.layers.Dense(128, activation = "relu"))
    model.add(tf.keras.layers.Dense(32, activation = "relu"))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model


def build_net_with_dropout(input_size):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(input_size,activation = 'relu',kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(128, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dropout(0.7))
    model.add(tf.keras.layers.Dense(32, activation = "relu",kernel_regularizer = tf.keras.regularizers.L1(0.01)))
    model.add(tf.keras.layers.Dense(1, activation = 'relu'))
    return model

def model_run(model,train_x,val_y, train_y, val_x, num_epochs):
    loss = MeanSquaredLogarithmicError()
    optimizer = Adam(learning_rate = 0.005)
    model.compile( loss= loss, optimizer = optimizer, metrics = ["msle"])
    #training
    history = model.fit(train_x,train_y, epochs = num_epochs,batch_size= 70, verbose = 1,validation_data = (val_x,val_y))
    return history

def plotter(history,metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric,'val_'+metric])
    plt.show()
    



In [ ]:
#putting it all together

ann_without = build_net_no_dropout(X_train.shape[1])
hist_ann_without = model_run(ann_without,X_train,Y_test,Y_train,X_test,30)
plotter(hist_ann_without, "msle")
plotter(hist_ann_without, "loss")


In [ ]:
ann_with = build_net_with_dropout(X_train.shape[1])
hist_ann_with = model_run(ann_with,X_train,Y_test,Y_train,X_test,40)
plotter(hist_ann_with, "msle")
plotter(hist_ann_with, "loss")

In [ ]:

Predictions = ann_without.predict(X_test)
Real = Y_test
dft = pd.DataFrame({"real_2017":list(Y_test),"Prediction_2017":list(Predictions)})
plt.figure(figsize =(20,4))
plt.plot(dft["real_2017"])
plt.plot(dft["Prediction_2017"])
plt.legend(["Real", "Predictions"])
plt.show()

In [6]:
# each additional component in the waste to energy supply chain will be represented 

class Component():
    def __init__(self, index, name, num, cap, points,k, pointx,data=biomass.copy(),distance = distance.copy(),refinery=False,spare =points.copy()):
        self.name = name
        self.index = index
        self.num = num       
        self.capacity = cap
        self.tree = rtree.index.Rtree()
        if refinery==True:
            self.point = spare.copy()[index]
            
        else:
            self.point = points[index]
            del(spare)
        for i,p in enumerate(pointx):
            self.tree.insert(i,p+p,p)
       
            
        
        # k nearest point
        self.sources= list(self.tree.nearest(self.point, k,objects='raw' ))
        #print(len(self.sources))
        self.sources_index = []
        
        for i in self.sources:
            if refinery ==False:
                self.sources_index.append(points.index(i))
                pointx.remove(i)
            elif refinery == True:
                self.sources_index.append(spare.index(i))
                pointx.remove(i)
                
        self.sources= self.sources[1:]
        self.sources_index= self.sources_index[1:]
        #print("Original number of sources {}".format(len(self.sources)))
        
        #this part calculates the total output of the sources  
        sources_output = data["2017"][self.sources_index].sum()
        
        self.under_util = 0
        if sources_output>self.capacity:
            while True:
                pointx.append(self.sources[-1])
                self.sources.pop()
                self.sources_index.pop()
                sources_output = data["2017"][self.sources_index].sum()
                
                if sources_output<=self.capacity:
                    self.under_util = self.capacity-sources_output
                    break
                else:
                    continue
           
        self.total_dist = 0
        for i in self.sources_index:
            self.total_dist+=distance.iloc[i,self.index]
        self.pointx = pointx[:]
        #print("final number of sources {}".format(len(self.sources)))
        
    def prevent_duplicates(self):
        return self.pointx
            
    def add(self,df):
        q= int(self.index)
        df.Type[q] = str(self.name)
        return df
    def cost(self):
        return self.total_dist,self.under_util
        



def make_components(number,name, k, cap,pointlist,pointx=[],index_=-1,refinery=False):
   
    components = [] 
    if 1==1:        
        listr = [] 
        pointx = pointlist[:]
        for i in range(number):
            index = random.randint(0,2417)
            if index not in listr :
                
                
                comp = Component(index,name,i,cap,pointlist,k,pointx,refinery = refinery)
                pointx= comp.prevent_duplicates()
                components.append(comp)
                listr.append(index)
                
            else:
                i = i-1
                continue
    print("done")
    return components,pointx
        
    
    
     
    
    


In [ ]:
refs=Component(90,"depo_location",90,2000,points,3,points[2:2220],refinery = False)
print(refs.index)

In [ ]:
#creating a custom environment with gym

class Environment(Env):
    def __init__(self,num_dep,num_refinery, points = points):
        a = 0.001
        b = 1
        c = 1
        self.k = 80
        self.points = points 
        self.num_dep = num_dep
        self.refinery_cap = 100000
        self.num_refinery = num_refinery
        self.dep_cap = 20000
        self.depots,self.pointx = make_components(num_dep,"depot_location",self.k, 20000,points)
        self.dep_points = [q.point for q in self.depots]
        self.refineries,self.refpointx = make_components(self.num_refinery,"refinery_location",int(num_dep//num_refinery), 100000,self.dep_points,refinery=True)
        self.ref_points = [q.point for q in self.refineries]
        self.transport = 0
        self.underutil = 0
        self.ndim=(self.num_dep+self.num_refinery,2)
        self.observation_space = Box(high= np.inf,low = -np.inf,shape = (1,))
        self.action_space =Discrete(np.prod(self.ndim))
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        for i in self.refineries:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        self.state = (a*self.transport)+(c*self.underutil)
        self.episode_len = 80
        self.previous_score = []
        self.reinitialize= {"depots":self.depots,
                            "refinery":self.refineries, 
                            "refinery_points":self.ref_points,
                            "depot_points" =self.dep_points,
                            "deppointx":self.pointx,
                            'refpoints':self.refpointx,
                            
                           }
            
    def observe(self):
        return np.array([self.state])
    
    def reset(self):
        ["depots"]:self.depots,
        ["refinery"] = self.refineries, 
        ["refinery_points"] =self.ref_points,
        ["depot_points"] =self.dep_points,
        ["deppointx"] = self.pointx,
        ['refpoints'] = self.refpointx
        self.previous_score = []
        self.recalc()
        obs = self.observe()
        return obs
        
    def recalc (self):
        a = 0.001
        b = 1
        c = 1
        for i in self.depots:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        for i in self.refineries:
            tr,un = i.cost()
            self.transport+=tr
            self.underutil+=un
        var = (a*self.transport)+(c*self.underutil)
        return var
        
    def close(self):
        pass
        
    def step(self, action):
        reward = 0
        mapping = tuple(np.ndindex(self.ndim))
        action = mapping[action]
        if action[0]>=(self.num_dep) and self.episode_len>0:
            # if the agent picks from [0, self.num_dep] that indicates a depot, else its a refinery
            # in this case it is a refinery that has been selected
            self.episode_len -= 1 
              #selcting one the final motion [left:0,right:1]                
            rp = self.ref_points[action[0]-self.num_dep]
            indx = self.points.index(rp)
            if action[1] == 0: #left
                if indx-10>=0:
                    for i in self.refineries[action[0]-self.num_dep].sources:
                        self.refpointx.append(i)
                        
                        
                    ref_new =Component(indx-10,"refinery_location",indx-10,100000,self.dep_points,4,self.refpointx,refinery = True)
                    self.refpointx = ref_new.prevent_duplicates()
                    self.refineries[action[0]-self.num_dep] = ref_new
                    self.ref_points[action[0]-self.num_dep] = ref_new.point 
                    self.state = self.recalc() 
                else:
                    reward -= 20
                    
            elif action[1] == 1:# right
                if indx+10<=2417:
                    for i in self.refineries[action[0]-self.num_dep].sources:
                        self.refpointx.append(i)
                    ref_new=Component(indx+10,"refinery_location",indx+10,100000,self.dep_points,4,self.refpointx,refinery = True)                     
                    self.refpointx = ref_new.prevent_duplicates()
                    print(type(ref_new))
                    self.refineries[action[0]-self.num_dep] =ref_new
                   
                    self.ref_points[action[0]-self.num_dep] =ref_new.point 
                    self.state = self.recalc()
                else:
                    reward -= 20      
            
        elif action[0]<(self.num_dep) and self.episode_len>0:                               #in this case the agent selects a depot
            self.episode_len -= 1 
              #selcting one the final motion [left,right]
            dp = self.dep_points[action[0]]
            indx = self.points.index(dp)
            if action[1] == 0: #left
                
                if indx-10>=0:
                    for i in self.depots[action[0]].sources:
                        self.pointx.append(i)
                    dep_new= Component(indx-10,"depot_location",indx-10,20000,self.points,self.k,self.pointx,refinery = False)
                    self.pointx=dep_new.prevent_duplicates() 
                    
                    self.depots[action[0]] =dep_new
                    self.dep_points[action[0]] =dep_new.point 
                    self.state = self.recalc() 
                else:
                    reward -= 20
                    
            elif action[1] == 1:# right
                if indx+10<=2417:
                    for i in self.depots[action[0]].sources:
                        self.pointx.append(i)
                    dep_new = Component(indx+10,"depot_location",indx+10,20000,self.points,self.k,self.pointx,refinery = False)                    
                    self.pointx=dep_new.prevent_duplicates() 
                    self.depots[action[0]] =dep_new
                    self.dep_points[action[0]] =dep_new.point 
                    self.state = self.recalc() 
                else:
                    reward -= 20
                    
                    
                
        
        # Calculating the reward
        if len(self.previous_score)>0:
            reward += (20*(self.previous_score[-1]-self.state))/self.previous_score[-1]
            self.previous_score.append(self.state)
 
  
        elif len(self.previous_score)==0:
            self.previous_score.append(self.state) 
            
             
        
        # Checking if episode is done
        if self.episode_len <= 0: 
            done = True
        else:
            done = False
        
        # Setting the placeholder for info
        info = {}
        
        # Returning the step information
        return self.observe(), reward, done,info

In [14]:
Enviro = Environment(25,5)
state = Enviro.observation_space.shape
actions = Enviro.action_space.n

done
done


In [ ]:
!pip install rl-agents==0.1.1


In [16]:
# model
model = tf.keras.models.Sequential()    
model.add(tf.keras.layers.Dense(32, activation='relu', input_shape =(1,1)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(actions, activation='linear'))


In [ ]:
model.summary()

In [11]:

def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [17]:
Agent = build_agent(model, actions)
Agent.compile(tf.keras.optimizers.legacy.Adam(learning_rate=1e-2), metrics=['mae'])
Agent.fit(Enviro , nb_steps=6000, visualize=False, verbose=1)

Training for 6000 steps ...
done
done
(1,)
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 30:02 - reward: 0.0000e+00

/opt/conda/lib/python3.10/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   23/10000 [..............................] - ETA: 6:05 - reward: -2.4115<class '__main__.Component'>
<class '__main__.Component'>
   39/10000 [..............................] - ETA: 5:35 - reward: -1.6392<class '__main__.Component'>
<class '__main__.Component'>
   60/10000 [..............................] - ETA: 5:37 - reward: -1.1888<class '__main__.Component'>
<class '__main__.Component'>
   79/10000 [..............................] - ETA: 5:43 - reward: -0.9726done
done
(1,)
  101/10000 [..............................] - ETA: 10:42 - reward: -1.2198

/opt/conda/lib/python3.10/site-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  160/10000 [..............................] - ETA: 10:15 - reward: -0.9342done
done
(1,)
  239/10000 [..............................] - ETA: 11:07 - reward: -0.9628done
done
(1,)
<class '__main__.Component'>
  241/10000 [..............................] - ETA: 12:57 - reward: -0.9553<class '__main__.Component'>
<class '__main__.Component'>
  276/10000 [..............................] - ETA: 12:02 - reward: -1.0565<class '__main__.Component'>
<class '__main__.Component'>
  292/10000 [..............................] - ETA: 11:59 - reward: -1.0411<class '__main__.Component'>
<class '__main__.Component'>
  319/10000 [..............................] - ETA: 11:35 - reward: -0.9883done
done
(1,)
<class '__main__.Component'>
  384/10000 [>.............................] - ETA: 12:01 - reward: -1.0453<class '__main__.Component'>
<class '__main__.Component'>
  400/10000 [>.............................] - ETA: 11:43 - reward: -1.0135done
done
(1,)
<class '__main__.Component'>
  401/10000 [>.......

IndexError: list index out of range